In [14]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
# Import Dataset from sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
from sklearn.datasets import load_iris # Load Iris Data

iris = load_iris() # Creating pd DataFrames

iris_df = pd.DataFrame(data= iris.data, columns= iris.feature_names)
target_df = pd.DataFrame(data= iris.target, columns= ['species'])

def converter(specie):
    if specie == 0:
        return 'setosa'
    elif specie == 1:
        return 'versicolor'
    else:
        return 'virginica'

target_df['species'] = target_df['species'].apply(converter)# Concatenate the DataFrames
iris_df = pd.concat([iris_df, target_df], axis= 1)

In [47]:
# Converting Objects to Numerical dtype
iris_df.drop('species', axis= 1, inplace= True)
target_df = pd.DataFrame(columns= ['species'], data= iris.target)
iris_df = pd.concat([iris_df, target_df], axis= 1)# Variables
X= iris_df.drop(labels= 'sepal length (cm)', axis= 1)
y= iris_df['sepal length (cm)']

# Splitting the Dataset 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.13, random_state= 101)

# Instantiating LinearRegression() Model
lr = LinearRegression()

# Training/Fitting the Model
lr.fit(X_train, y_train)

# Making Predictions
lr.predict(X_test)
pred = lr.predict(X_test)

# Evaluating Model's Performance
print('Mean Squared Error:', mean_squared_error(y_test, pred))


Mean Squared Error: 0.1123029571353145


In [ ]:
print(X.values[:].max(), X.values[:].min())
print(y.values[:].max(), y.values[:].min())

In [48]:
W = lr.coef_
b = lr.intercept_

In [49]:
Y_expected = np.matmul(X_test, W) + b

assert (Y_expected == lr.predict(X_test)).all()

In [60]:
print(X.values[:].max(), X.values[:].min())
print(y.values[:].max(), y.values[:].min())
print(np.matmul(X_test, W).max(), np.matmul(X_test, W).min())
print((y_test - pred).max(), (y_test - pred).min())
print(((y_test - pred) ** 2).max(), ((y_test - pred) ** 2).min())
print(b)

6.9 0.0
7.9 4.3
5.558081705128363 2.9133892796758256
0.5143332457856635 -0.553112438773117
0.3059333699255451 0.000844615740033696
1.7620311370254882


In [62]:
alpha_X = 0
beta_X = 8
alpha_Y = 2
beta_Y = 8
alpha_Yt = 2
beta_Yt = 8
alpha_b = 0
beta_b = 8
alpha_R = -1
beta_R = 1
alpha_S = 0
beta_S = 0.4

In [51]:
X_test.shape

(20, 4)